# Chapter 1: Set up data

Before executing the chapters that follow, some input datasets need to be downloaded to the data directory from their sources, and some need to be modified slightly. To do this, please follow the steps below.

## Get data

### 1. Download the Cancer Cell Line Encyclopedia (CCLE) datasets from the CCLE web site

Watch [this quick video](https://www.youtube.com/watch?v=iTFm3n_JRw8&feature=youtu.be) or follow these steps:

1. Go [here](https://portals.broadinstitute.org/ccle_legacy/toa/termsOfAccess/20/) to begin making an account with CCLE.
2. Click "Accept" and enter your account information
3. Go the the "Browse" tab and select "Data"
4. At the bottom of the page beneath "Publication specific data files" click"Show available data"
5. Click "CCLE_MUT_EXPR_RPPA_OncoGPS.zip" to download
6. Unzip "CCLE_MUT_EXPR_RPPA_OncoGPS.zip" and move the 4 files inside to the onco-gps-paper-analysis/data directory.

### 2. Download the gene dependencies (RNAi) dataset from the Achilles project web site

Watch [this quick video](https://www.youtube.com/watch?v=wj0cJC9-XYw&feature=youtu.be) or follow these steps:

1. Go [here](https://portals.broadinstitute.org/achilles/users/sign_up) to make an account with Project Achilles.
2. Confirm your email with the confirmation link in an email you'll receive from Project Achilles.
3. Go [here](https://portals.broadinstitute.org/achilles/datasets/15/download) and click the "ExpandedGeneZSolsCleaned.csv" link to download the gene dependencies dataset
4. Move the downloaded "ExpandedGeneZSolsCleaned.csv" file to the onco-gps-paper-analysis/data directory

Now, run the cell below to rename the dataset.

### 3. Download the CTRP v2 datasets from the Broad Institute

1. Go here: ftp://caftpd.nci.nih.gov/pub/OCG-DCC/CTD2/Broad/CTRPv2.2_2015_pub_CancerDisc_5_1210/ 
2. Click "CTRPv2.2_2015_pub_CancerDisc_5_1210.zip" to download the CTRP dataset
3. Move the downloaded "CTRPv2.2_2015_pub_CancerDisc_5_1210.zip" file to the onco-gps-analysis-paper/data directory
4. Run the cell below to prepare the CTRP dataset for analysis.

## Prepare data

Run the cells below.

### 1. Set up notebook and import [CCAL](https://github.com/KwatME/ccal)

In [5]:
from notebook_environment import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. Unzip and prepare datasets

In [6]:
df = pd.read_csv('../data/ExpandedGeneZSolsCleaned.csv', index_col=0)
ccal.write_gct(df, '../data/achilles__gene_x_ccle_cellline.gct')

ccal.unzip('../data/CTRPv2.2_2015_pub_CancerDisc_5_1210.zip')
ccal.unzip('../data/gene_set__gene_set_x_ccle_cellline.gct.zip')

# Read compound data
auc = pd.read_table('../data/v22.data.auc_sensitivities.txt')
print(auc.shape)

cpd = pd.read_table('../data/v22.meta.per_compound.txt', index_col=0)
print(cpd.shape)

ccl = pd.read_table('../data/v22.meta.per_cell_line.txt', index_col=0)
print(ccl.shape)

# Make dict for faster ID-to-name look up
cpd_d = cpd['cpd_name'].to_dict()
ccl_d = ccl['ccl_name'].to_dict()

# Make empty compound-x-cellline matrix
compound_x_cellline = pd.DataFrame(
    index=sorted(set(cpd['cpd_name'])), columns=sorted(set(ccl['ccl_name'])))
print(compound_x_cellline.shape)

# Populate compound-x-cellline matrix
for i, (i_cpd, i_ccl, a) in auc.iterrows():

    # Get compound name
    cpd_n = cpd_d[i_cpd]

    # Get cellline name
    ccl_n = ccl_d[i_ccl]

    # Get current AUC
    a_ = compound_x_cellline.loc[cpd_n, ccl_n]

    # If the current AUC is not set, set with this AUC
    if pd.isnull(a_):
        compound_x_cellline.loc[cpd_n, ccl_n] = a

    # If this AUC is smaller than the current AUC, set with this AUC
    elif a < a_:

        print('Updating AUC of compound {} on cellline {}: {:.3f} ==> {:.3f}'.
              format(cpd_n, ccl_n, a_, a))

        compound_x_cellline.loc[cpd_n, ccl_n] = a

# Update cellline names to match CCLE cellline names
columns = list(compound_x_cellline.columns)

# Read CCLE cellline annotations
a = pd.read_table('../data/CCLE_sample_info_file_2012-10-18.txt', index_col=0)

# Get CCLE cellline names
for i, ccl_n in enumerate(compound_x_cellline.columns):

    matches = []

    for ccle_n in a.index:
        if ccl_n.lower() == ccle_n.lower().split('_')[0]:
            matches.append(ccle_n)

    if 0 == len(matches):
        print('0 match: {}; matching substring ...'.format(ccl_n))

        for ccle_n in a.index:

            if ccl_n.lower() in ccle_n.lower():

                print('\t{} ==> {}.'.format(ccl_n, ccle_n))
                matches.append(ccle_n)

    if 1 == len(matches):

        print('{} ==> {}.'.format(ccl_n, matches[0]))
        columns[i] = matches[0]

    else:
        print('1 < matches: {} ==> {}'.format(ccl_n, matches))

# Update with CCLE cellline names
compound_x_cellline.columns = columns

# Write .gct file
ccal.write_gct(compound_x_cellline,
               '../data/ctd2__compound_x_ccle_cellline.gct')

compound_x_cellline

(260496, 3)
(481, 9)
(664, 8)
(481, 645)
Updating AUC of compound ML311 on cellline CCFSTTG1: 15.000 ==> 14.352
Updating AUC of compound ML311 on cellline NCIH460: 13.004 ==> 12.245
Updating AUC of compound ML311 on cellline IGROV1: 13.147 ==> 12.092
Updating AUC of compound ML311 on cellline OAW28: 13.205 ==> 11.837
Updating AUC of compound ML311 on cellline ASPC1: 13.483 ==> 13.200
Updating AUC of compound ML311 on cellline CAL51: 12.028 ==> 11.655
Updating AUC of compound ML311 on cellline NCIH1869: 14.149 ==> 13.606
Updating AUC of compound ML311 on cellline LOXIMVI: 13.386 ==> 12.332
Updating AUC of compound ML311 on cellline MKN74: 13.969 ==> 12.234
Updating AUC of compound zebularine on cellline CCFSTTG1: 14.999 ==> 14.634
Updating AUC of compound zebularine on cellline A549: 14.507 ==> 13.312
Updating AUC of compound zebularine on cellline NCIH460: 14.581 ==> 12.742
Updating AUC of compound zebularine on cellline ASPC1: 15.699 ==> 13.944
Updating AUC of compound zebularine on c

Updating AUC of compound paclitaxel on cellline ASPC1: 9.750 ==> 9.291
Updating AUC of compound paclitaxel on cellline CAL51: 9.707 ==> 5.014
Updating AUC of compound paclitaxel on cellline A375: 11.572 ==> 7.637
Updating AUC of compound paclitaxel on cellline LOXIMVI: 14.997 ==> 7.762
Updating AUC of compound paclitaxel on cellline MKN74: 9.152 ==> 6.965
Updating AUC of compound hyperforin on cellline CCFSTTG1: 15.447 ==> 14.839
Updating AUC of compound hyperforin on cellline NCIH1299: 15.580 ==> 14.640
Updating AUC of compound hyperforin on cellline A375: 18.677 ==> 14.182
Updating AUC of compound hyperforin on cellline NCIH1869: 14.920 ==> 14.371
Updating AUC of compound hyperforin on cellline MKN74: 14.038 ==> 12.211
Updating AUC of compound brefeldin A on cellline CCFSTTG1: 14.679 ==> 12.948
Updating AUC of compound brefeldin A on cellline OAW28: 9.371 ==> 7.765
Updating AUC of compound brefeldin A on cellline SUIT2: 9.336 ==> 9.333
Updating AUC of compound brefeldin A on cellline

Updating AUC of compound PX-12 on cellline CCFSTTG1: 15.116 ==> 14.770
Updating AUC of compound PX-12 on cellline A549: 14.621 ==> 14.587
Updating AUC of compound PX-12 on cellline NCIH520: 14.270 ==> 13.087
Updating AUC of compound PX-12 on cellline IGROV1: 14.234 ==> 13.212
Updating AUC of compound PX-12 on cellline OAW28: 14.310 ==> 12.950
Updating AUC of compound PX-12 on cellline LOXIMVI: 14.278 ==> 13.498
Updating AUC of compound PX-12 on cellline KE39: 14.126 ==> 13.446
Updating AUC of compound PX-12 on cellline MKN74: 14.744 ==> 14.205
Updating AUC of compound PD318088 on cellline CCFSTTG1: 14.840 ==> 14.451
Updating AUC of compound PD318088 on cellline A549: 11.127 ==> 9.559
Updating AUC of compound PD318088 on cellline NCIH460: 14.742 ==> 14.265
Updating AUC of compound PD318088 on cellline IGROV1: 11.728 ==> 9.794
Updating AUC of compound PD318088 on cellline OAW28: 13.666 ==> 11.608
Updating AUC of compound PD318088 on cellline CAL51: 14.591 ==> 13.512
Updating AUC of compo

Updating AUC of compound fulvestrant on cellline SUIT2: 14.637 ==> 14.606
Updating AUC of compound BRD-A86708339 on cellline CCFSTTG1: 15.157 ==> 13.818
Updating AUC of compound BRD-A86708339 on cellline NCIH1869: 11.099 ==> 9.681
Updating AUC of compound BRD-A86708339 on cellline SKUT1: 10.673 ==> 8.740
Updating AUC of compound CID-5951923 on cellline NCIH1299: 15.854 ==> 15.721
Updating AUC of compound CID-5951923 on cellline NCIH460: 14.711 ==> 14.060
Updating AUC of compound CID-5951923 on cellline ASPC1: 14.094 ==> 13.453
Updating AUC of compound CID-5951923 on cellline CAL51: 16.163 ==> 15.276
Updating AUC of compound CID-5951923 on cellline NCIH1869: 14.176 ==> 13.611
Updating AUC of compound CID-5951923 on cellline MKN74: 14.649 ==> 14.581
Updating AUC of compound CID-5951923 on cellline SKUT1: 14.646 ==> 14.587
Updating AUC of compound FQI-1 on cellline NCIH460: 14.086 ==> 13.842
Updating AUC of compound FQI-1 on cellline IGROV1: 13.281 ==> 12.697
Updating AUC of compound FQI-

Updating AUC of compound QS-11 on cellline AGS: 16.464 ==> 14.627
Updating AUC of compound AT-406 on cellline CCFSTTG1: 14.762 ==> 14.484
Updating AUC of compound AT-406 on cellline A549: 14.299 ==> 13.299
Updating AUC of compound AT-406 on cellline NCIH520: 14.112 ==> 11.507
Updating AUC of compound AT-406 on cellline IGROV1: 16.035 ==> 14.843
Updating AUC of compound AT-406 on cellline LOXIMVI: 15.256 ==> 11.978
Updating AUC of compound SU11274 on cellline A549: 14.070 ==> 13.611
Updating AUC of compound SU11274 on cellline IGROV1: 14.074 ==> 12.604
Updating AUC of compound SU11274 on cellline OAW28: 14.266 ==> 14.230
Updating AUC of compound SU11274 on cellline ASPC1: 16.544 ==> 15.136
Updating AUC of compound SU11274 on cellline SUIT2: 15.025 ==> 14.042
Updating AUC of compound SU11274 on cellline CAL51: 13.336 ==> 12.741
Updating AUC of compound SU11274 on cellline A375: 15.000 ==> 13.861
Updating AUC of compound SU11274 on cellline NCIH1869: 15.000 ==> 14.302
Updating AUC of comp

...

Updating AUC of compound PL-DI on cellline NCIH1869: 11.714 ==> 11.696
Updating AUC of compound PL-DI on cellline LOXIMVI: 10.673 ==> 10.006
Updating AUC of compound PL-DI on cellline MKN74: 11.367 ==> 10.876
Updating AUC of compound skepinone-L on cellline CCFSTTG1: 14.807 ==> 14.362
Updating AUC of compound skepinone-L on cellline ASPC1: 14.806 ==> 14.420
Updating AUC of compound skepinone-L on cellline NCIH1869: 14.565 ==> 14.546
Updating AUC of compound skepinone-L on cellline SKUT1: 13.820 ==> 13.421
Updating AUC of compound nelarabine on cellline CCFSTTG1: 15.527 ==> 14.890
Updating AUC of compound nelarabine on cellline NCIH1299: 15.544 ==> 15.248
Updating AUC of compound nelarabine on cellline A549: 14.374 ==> 13.655
Updating AUC of compound nelarabine on cellline NCIH460: 15.011 ==> 13.261
Updating AUC of compound nelarabine on cellline ASPC1: 15.809 ==> 14.176
Updating AUC of compound nelarabine on cellline CAL51: 15.246 ==> 14.902
Updating AUC of compound nelarabine on celll

Updating AUC of compound momelotinib on cellline LOXIMVI: 12.140 ==> 11.913
Updating AUC of compound momelotinib on cellline DU145: 12.750 ==> 12.374
Updating AUC of compound momelotinib on cellline MKN74: 12.496 ==> 11.711
Updating AUC of compound momelotinib on cellline SKUT1: 13.240 ==> 13.129
Updating AUC of compound istradefylline on cellline CCFSTTG1: 16.305 ==> 14.732
Updating AUC of compound istradefylline on cellline ASPC1: 15.742 ==> 14.283
Updating AUC of compound istradefylline on cellline NCIH1869: 14.481 ==> 14.275
Updating AUC of compound fingolimod on cellline NCIH460: 12.981 ==> 12.836
Updating AUC of compound fingolimod on cellline IGROV1: 12.607 ==> 11.923
Updating AUC of compound fingolimod on cellline NCIH1869: 11.965 ==> 11.941
Updating AUC of compound fingolimod on cellline SKUT1: 13.155 ==> 12.632
Updating AUC of compound bortezomib on cellline CCFSTTG1: 13.104 ==> 12.744
Updating AUC of compound bortezomib on cellline NCIH1299: 13.193 ==> 11.994
Updating AUC of

Updating AUC of compound GSK461364 on cellline A549: 6.725 ==> 4.181
Updating AUC of compound GSK461364 on cellline NCIH460: 9.742 ==> 8.819
Updating AUC of compound GSK461364 on cellline IGROV1: 7.396 ==> 6.293
Updating AUC of compound GSK461364 on cellline OAW28: 12.427 ==> 8.454
Updating AUC of compound GSK461364 on cellline CAL51: 5.836 ==> 3.334
Updating AUC of compound GSK461364 on cellline A375: 11.852 ==> 5.731
Updating AUC of compound GSK461364 on cellline NCIH1869: 11.197 ==> 11.101
Updating AUC of compound GSK461364 on cellline LOXIMVI: 6.277 ==> 5.862
Updating AUC of compound GSK461364 on cellline KE39: 12.227 ==> 10.315
Updating AUC of compound GSK461364 on cellline MKN74: 9.813 ==> 6.785
Updating AUC of compound bexarotene on cellline CCFSTTG1: 15.203 ==> 15.145
Updating AUC of compound bexarotene on cellline A549: 14.992 ==> 12.288
Updating AUC of compound bexarotene on cellline NCIH1299: 16.665 ==> 15.055
Updating AUC of compound bexarotene on cellline OAW28: 16.266 ==>

Updating AUC of compound VAF-347 on cellline CCFSTTG1: 15.000 ==> 14.893
Updating AUC of compound VAF-347 on cellline NCIH1869: 14.879 ==> 13.386
Updating AUC of compound VAF-347 on cellline LOXIMVI: 15.065 ==> 14.442
Updating AUC of compound VAF-347 on cellline MKN74: 15.448 ==> 14.062
Updating AUC of compound VAF-347 on cellline SKUT1: 15.000 ==> 14.848
Updating AUC of compound pifithrin-mu on cellline A549: 15.495 ==> 14.473
Updating AUC of compound pifithrin-mu on cellline NCIH460: 15.344 ==> 14.878
Updating AUC of compound pifithrin-mu on cellline NCIH520: 14.758 ==> 12.998
Updating AUC of compound pifithrin-mu on cellline IGROV1: 16.408 ==> 14.457
Updating AUC of compound pifithrin-mu on cellline OAW28: 15.486 ==> 13.497
Updating AUC of compound pifithrin-mu on cellline ASPC1: 15.918 ==> 14.994
Updating AUC of compound pifithrin-mu on cellline NCIH1869: 14.523 ==> 14.460
Updating AUC of compound pifithrin-mu on cellline LOXIMVI: 14.868 ==> 14.453
Updating AUC of compound pifithri

,22RV1_PROSTATE,2313287_STOMACH,253J_URINARY_TRACT,253JBV_URINARY_TRACT,42MGBA_CENTRAL_NERVOUS_SYSTEM,5637_URINARY_TRACT,639V_URINARY_TRACT,647V_URINARY_TRACT,769P_KIDNEY,786O_KIDNEY,...,WM983B_SKIN,YAPC_PANCREAS,YD10B_UPPER_AERODIGESTIVE_TRACT,YD15_SALIVARY_GLAND,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YH13_CENTRAL_NERVOUS_SYSTEM,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST,ZR7530_BREAST
16-beta-bromoandrosterone,14.565,12.668,14.839,14.586,14.917,14.14,15.446,14.585,14.252,14.749,...,14.607,15.524,16.285,16.173,15,14.468,15.064,NaN,15.597,13.764
"1S,3R-RSL-3",9.2997,10.231,7.5951,15,6.2174,7.7638,13.301,8.0628,7.5991,5.9814,...,11.351,13.108,11.267,11.708,9.6433,5.7127,7.0035,10.699,12.43,6.2003
3-Cl-AHPC,10.259,10.418,11.077,10.379,8.9201,9.3751,10.052,9.3562,9.5917,10.551,...,11.452,12.207,9.949,9.5841,11.884,12.84,11.865,13.934,14.273,11.769
968,14.376,15.11,NaN,13.36,NaN,14.045,15.448,NaN,14.915,NaN,...,NaN,NaN,16.233,15.618,15.383,NaN,NaN,NaN,NaN,NaN
A-804598,NaN,NaN,NaN,NaN,14.147,NaN,NaN,15.312,NaN,14.498,...,14.278,15,NaN,NaN,NaN,14.548,14.678,NaN,15,14.074
AA-COCF3,12.043,9.3923,12.875,12.806,14.543,12.344,12.671,15.486,13.28,14.793,...,13.122,15.286,16.419,15,13.769,12.527,15.328,10.381,15,11.939
ABT-199,NaN,NaN,NaN,NaN,15.172,NaN,NaN,15.126,NaN,14.608,...,13.041,15.729,NaN,NaN,NaN,14.539,15.654,15.115,15,14.683
ABT-737,12.965,14.623,13.252,13.602,14.98,12.746,15.739,NaN,13.42,12.96,...,13.442,15,14.638,14.698,14.806,NaN,NaN,14.809,14.476,14.405
AC55649,13.845,13.779,14.604,14.805,NaN,13.304,15.679,15,14.751,14.746,...,14.228,15,13.788,15.787,15,14.144,14.474,14.691,15.366,14.76
AGK-2,15.266,16.757,15.176,17.642,10.679,15,14.703,15,15.958,14.633,...,13.522,15.572,13.647,14.184,15.121,14.326,16.055,16.273,15,14.676


### 3. Check that all datasets exist

If you get an error running the cell below, get the dataset the error says you're missing, and run it again.

In [7]:
for fn in [
        'gene_x_kras_isogenic_and_imortalized_celllines.gct',
        'mutation__gene_x_ccle_cellline.gct',
        'rpkm__gene_x_ccle_cellline.gct',
        'gene_set__gene_set_x_ccle_cellline.gct',
        'regulator__gene_set_x_ccle_cellline.gct',
        'rppa__protein_x_ccle_cellline.gct',
        'achilles__gene_x_ccle_cellline.gct',
        'ctd2__compound_x_ccle_cellline.gct',
        'annotation__feature_x_ccle_cellline.gct',
]:
    assert fn in os.listdir('../data/'), 'Missing {}!'.format(fn)

### 4. Make the CCLE data object

In [8]:
# Make the CCLE data object used in coming chapters.

ccle = {
    'Mutation': {
        'df': ccal.read_gct('../data/mutation__gene_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'binary'
    },
    'Gene Expression': {
        'df': ccal.read_gct('../data/rpkm__gene_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Gene Set': {
        'df': ccal.read_gct('../data/gene_set__gene_set_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Regulator Gene Set': {
        'df': ccal.read_gct('../data/regulator__gene_set_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Protein Expression': {
        'df': ccal.read_gct('../data/rppa__protein_x_ccle_cellline.gct'),
        'emphasis': 'high',
        'data_type': 'continuous'
    },
    'Gene Dependency (Achilles)': {
        'df': ccal.read_gct('../data/achilles__gene_x_ccle_cellline.gct'),
        'emphasis': 'low',
        'data_type': 'continuous'
    },
    'Drug Sensitivity (CTD^2)': {
        'df': ccal.read_gct('../data/ctd2__compound_x_ccle_cellline.gct'),
        'emphasis': 'low',
        'data_type': 'continuous'
    },
    'Primary Site': {
        'df':
        ccal.make_membership_df_from_categorical_series(
            ccal.read_gct('../data/annotation__feature_x_ccle_cellline.gct')
            .loc['Site Primary']),
        'emphasis':
        'high',
        'data_type':
        'binary'
    }
}

with gzip.open('../data/ccle.pickle.gz', 'wb') as f:

    pickle.dump(ccle, f)

### Go to the [next chapter(2)](2 Generate oncogenic-activation signature.ipynb)